In [ ]:
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten , Conv2D, MaxPool2D
import matplotlib.pyplot as plt

In [ ]:
img2 = cv2.imread('../content/Image_test/2/AE7B30CA0F_0.258_2.jpg')
img2 = cv2.cvtColor(img2 , cv2.COLOR_BGR2RGB)

img3 = cv2.imread('../content/Image_test/3/AC3E812213_0.213_3.jpg')
img3 = cv2.cvtColor(img3 , cv2.COLOR_BGR2RGB)

img4 = cv2.imread('../content/Image_test/4/AE7B30C40E_0.194_4.jpg')
img4 = cv2.cvtColor(img4 , cv2.COLOR_BGR2RGB)

img5 = cv2.imread('../content/Image_test/5/ACA7B2A921_0.124_5.jpg')
img5 = cv2.cvtColor(img5 , cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12,4))
plt.subplot(141),plt.imshow(img2)
plt.subplot(142),plt.imshow(img3)
plt.subplot(143),plt.imshow(img4)
plt.subplot(144),plt.imshow(img5)

In [ ]:
width = 128
num_classes = 4
trainpath = '../content/Image_test/'
testpath = '../content/Image_wo1&6/test/'
trainImg = [trainpath+f for f in listdir(trainpath)]
testImg = [testpath+f for f in listdir(testpath)]

print("Train Images:")
print(trainImg)
print("\nTest Images:")
print(testImg)

In [ ]:
def img2data(path):
    rawImgs = []
    labels = []

    for imagePath in (path):
        for item in tqdm(listdir(imagePath)):
            file = join(imagePath, item)
            if file[-1] =='g':
                img = cv2.imread(file , cv2.COLOR_BGR2RGB)
                img = cv2.resize(img ,(width,width))
                rawImgs.append(img)

                l = imagePath.split('/')[-1]
                k = l.split('_')[-1]
                type_img = k.split('.')[0]

                if type_img == '2':
                    labels.append([1, 0, 0, 0])
                elif type_img == '3':
                    labels.append([0, 1, 0, 0])
                elif type_img == '4':
                    labels.append([0, 0, 1, 0])
                elif type_img == '5':
                    labels.append([0, 0, 0, 1])

    return rawImgs, labels, len(rawImgs)

In [ ]:
print('Data use to train :')
x_train, y_train, len_train = img2data(trainImg)

print('Data use to test :')
x_test, y_test, len_test = img2data(testImg)

In [ ]:
print("Number of data points in the training dataset (len_train):", len_train)
print("Number of data points in the testing dataset (len_test):", len_test)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
x_train.shape,y_train.shape,x_test.shape, y_test.shape

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3),
                                                include_top=False,
                                                weights='imagenet')
base_model.trainable = False
num_classes = 4

model = Sequential([
        base_model,

        Conv2D(128, 3, activation='relu'),
        Dense(16),
        Flatten(),
        Dense(num_classes, activation='softmax')
    ])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics= ['accuracy'])
batch_size = 32
# 24 28 30 31 29
epochs = 10

In [ ]:
history = model.fit(x_train, y_train ,batch_size=batch_size, epochs=epochs ,validation_data=(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
def preprocessing(file):
    img = cv2.imread(file, cv2.COLOR_BGR2RGB)
    ori = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (width, width))
    rimg = np.array(img)
    rimg = rimg.astype('float32')
    rimg /= 255
    rimg = np.reshape(rimg, (1, 128, 128, 3))
    #fft add preprocessing
    return rimg, ori

In [ ]:
def calculate_accuracy(type_accuracy, correct_predictions, total_predictions, label, item, model, file):
    real_label = item.split('_')[-1].split('.')[0]
    pre_img, ori = preprocessing(file)
    predict = model.predict(pre_img)
    result = label[np.argmax(predict)]

    if real_label == result:
        correct_predictions += 1

        if real_label in type_accuracy:
            type_accuracy[real_label][0] += 1  # Correct predictions
        else:
            type_accuracy[real_label] = [1, 0]  # [Correct predictions, Total predictions]

    if real_label in type_accuracy:
        type_accuracy[real_label][1] += 1  # Total predictions
    else:
        type_accuracy[real_label] = [0, 1]  # [Correct predictions, Total predictions]

    total_predictions += 1
    return type_accuracy, correct_predictions, total_predictions

In [ ]:
testpath = '../content/Image_wo1&6/test/'
testImg = [testpath + f for f in listdir(testpath) if listdir(join(testpath, f))]

type_accuracy = {}  # Dictionary to store accuracy for each type
correct_predictions = 0
total_predictions = 0
label = ['2', '3', '4', '5']

for imagePath in testImg:
    for item in listdir(imagePath):
        file = join(imagePath, item)
        if item.split('.')[0] != "":
            
            pre_img, ori = preprocessing(file)
            predict = model.predict(pre_img)
            
            result = label[np.argmax(predict)] # choose highest value in array of predict
            value = item.split('_')[1]
            print(predict)
            print('value    :  ' + str(value))
            print('real     :  ' + str(item.split('_')[-1].split('.')[0]))
            print('predict  :  ' + str(result))

            # Check accuracy
            real_label = item.split('_')[-1].split('.')[0]
            if real_label == result:
                correct_predictions += 1
                if real_label in type_accuracy:
                    type_accuracy[real_label][0] += 1  # Correct predictions
                else:
                    type_accuracy[real_label] = [1, 0]  # [Correct predictions, Total predictions]
            if real_label in type_accuracy:
                type_accuracy[real_label][1] += 1  # Total predictions
            else:
                type_accuracy[real_label] = [0, 1]  # [Correct predictions, Total predictions]
            total_predictions += 1

            plt.imshow(ori)
            plt.show()
            

In [ ]:
# Calculate and print the overall accuracy with 2 decimal places
overall_accuracy = correct_predictions / total_predictions
print("Overall Accuracy: {:.3f}".format(overall_accuracy))

# Calculate and print accuracy for each type with 2 decimal places
for type_label, (correct, total) in type_accuracy.items():
    type_acc = correct / total
    print("Accuracy for Type {}: {:.3f}".format(type_label, type_acc))


In [ ]:
model.save('../trained/model.h5')

In [ ]:
testpath = '../content/Image_wo1&6/test/'
testImg = [testpath + f for f in listdir(testpath) if listdir(join(testpath, f))]

type_accuracy = {}  # Dictionary to store accuracy for each type
correct_predictions = 0
total_predictions = 0

for imagePath in testImg:
    for item in listdir(imagePath):
        file = join(imagePath, item)
        if item.split('.')[0] != "":
            img = cv2.imread(file, cv2.COLOR_BGR2RGB)
            ori = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (width, width))
            rimg = np.array(img)
            rimg = rimg.astype('float32')
            rimg /= 255
            rimg = np.reshape(rimg, (1, 128, 128, 3))
            predict = model.predict(rimg)
            label = ['2', '3', '4', '5']
            result = label[np.argmax(predict)] # choose highest value in array of predict
            value = item.split('_')[1]
            print(predict)
            print('value    :  ' + str(value))
            print('real     :  ' + str(item.split('_')[-1].split('.')[0]))
            
            sub_label = ['2','3','4','5','6','7','8','9']
            
            if result == '2' :
                if value > '0.2' :
                    print('predict  :  ' + str(2))
                else :
                    print('predict  :  ' + str(3))
            elif result == '3' :
                if value > '0.2' :
                    print('predict  :  ' + str(4))
                else :
                    print('predict  :  ' + str(5))
            elif result == '4' :
                if value > '0.2' :
                    print('predict  :  ' + str(6))
                else :
                    print('predict  :  ' + str(7))
            elif result == '5' :
                if value > '0.2' :
                    print('predict  :  ' + str(8))
                else :
                    print('predict  :  ' + str(9))

            # Check accuracy
            real_label = item.split('_')[-1].split('.')[0]
            if real_label == result:
                correct_predictions += 1
                if real_label in type_accuracy:
                    type_accuracy[real_label][0] += 1  # Correct predictions
                else:
                    type_accuracy[real_label] = [1, 0]  # [Correct predictions, Total predictions]
            if real_label in type_accuracy:
                type_accuracy[real_label][1] += 1  # Total predictions
            else:
                type_accuracy[real_label] = [0, 1]  # [Correct predictions, Total predictions]
            total_predictions += 1

            plt.imshow(ori)
            plt.show()
